# Konfig - Demo



`konfig` is a wrapper around `ml_collection.ConfigDict`.

It allows to turn any Python function call /class construction into a nested dict which can be serialized/deserialized.

This unlocks (among others) type checking and autocomplete inside configuration files.

## Imports

In [ ]:
from etils.lazy_imports import *

Import `konfig`:

In [ ]:
with ecolab.adhoc():
  from kauldron import konfig

## End-to-end example

Here is a short example demonstrating how `konfig` work:

In [ ]:
# Step 1: Import your modules you want to be configurable (can be any module)
with konfig.imports():
  from flax import linen as nn

# Step 2: Create your configuration. Nothing is executed yet.
model_cfg = nn.Dense(features=32)  # This create a `ConfigDict` instead

# Step 3: The config object can be mutated, serialized to json,...
model_cfg.features = 64
model_cfg.use_bias = True

# Step 4: Resolve the config
# This is where the actual Python object are created (here the flax module)
model = konfig.resolve(model_cfg)

## Basic usage

### Creating the config

To turn any Python call into a configurable, the imports have to be wrapped inside a `konfig.imports()` contextmanager:

In [ ]:
with konfig.imports():
  import pathlib
  import optax
  from flax import linen as nn
  import numpy as np

Afterward, calling the functions won't execute anything yet, but instead create proxy `ConfigDict`:

In [ ]:
cfg = konfig.ConfigDict()

cfg.model = nn.Dense(features=32, dtype=np.float32)  # Create `ConfigDict`

cfg.optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adamw(
        learning_rate=0.01,
    ),
)

cfg.root_dir = pathlib.Path('a', 'b')

cfg

ConfigDict(
    model=nn.Dense(
        dtype=np.float32,
        features=32,
    ),
    optimizer=optax.chain(
        optax.clip_by_global_norm(1.0),
        optax.adamw(learning_rate=0.01),
    ),
    root_dir=pathlib.Path('a', 'b'),
)

`nn.Dense` looks like a standard Python call, but instead of instantiating the module, it creates a nested `ConfigDict` (which can be mutated, serialized, ...).

In [ ]:
model = nn.Dense(features=32, dtype=np.float32)

assert model == konfig.ConfigDict({
    '__qualname__': 'flax.linen:Dense',
    'features': 32,
    'dtype': {'__const__': 'numpy:float32'},
})

### Mutating the config

Indidivual attributes can be accessed/modified:

* kwargs - by mutating `cfg.kwarg_name = `:

In [ ]:
cfg.model.features = 64
cfg.model.use_bias = True
cfg.model

flax.linen.Dense(
    dtype=np.float32,
    features=64,
    use_bias=True,
)

* args - by mutating `cfg[arg_id] = `:

In [ ]:
cfg.root_dir[0] = 'root'
cfg.root_dir[-1] = 'last'
cfg.root_dir[2] = 'append'  # Append an additional argument
cfg.root_dir

pathlib.Path(
    'root',
    'last',
    'append',
)

* class/function - by mutating `cfg.__qualname__ =`:

In [ ]:
cfg.model.__qualname__ = 'flax.linen.DenseGeneral'
cfg.model

flax.linen.DenseGeneral(
    dtype=np.float32,
    features=64,
    use_bias=True,
)

* constants - by mutating `cfg.__const__ =`

In [ ]:
cfg.model.dtype.__const__ = 'numpy:float64'

### Resolving the config

Finally, use `konfig.resolve` to convert the config to actual object:

In [ ]:
resolved_cfg = konfig.resolve(cfg)
resolved_cfg

ConfigDict(
    model=DenseGeneral(
        # attributes
        features = 64
        axis = -1
        batch_dims = ()
        use_bias = True
        dtype = float64
        param_dtype = float32
        kernel_init = init
        bias_init = zeros
        precision = None
        dot_general = dot_general
    ),
    optimizer=(
        <function chain.<locals>.init_fn at 0x7f913bec3400>,
        <function chain.<locals>.update_fn at 0x7f913bec30a0>,
    ),
    root_dir=PosixPath('root/last/append'),
)

In [ ]:
resolved_cfg.root_dir.exists()  # root_dir is now a `pathlib` object

False

### Serializing/deserializing the config

ConfigDict can be serialized with `.to_json`

In [ ]:
json_str = cfg.to_json()
json_str

'{"model": {"__qualname__": "flax.linen.DenseGeneral", "features": 64, "dtype": {"__const__": "numpy:float64"}, "use_bias": true}, "optimizer": {"__qualname__": "optax:chain", "0": {"__qualname__": "optax:clip_by_global_norm", "0": 1.0}, "1": {"__qualname__": "optax:adamw", "learning_rate": 0.01}}, "root_dir": {"__qualname__": "pathlib:Path", "0": "root", "1": "last", "2": "append"}}'

And restored by passing the json dict:

In [ ]:
konfig.ConfigDict(json.loads(json_str))

ConfigDict(
    model=flax.linen.DenseGeneral(
        dtype=np.float64,
        features=64,
        use_bias=True,
    ),
    optimizer=optax.chain(
        optax.clip_by_global_norm(1.0),
        optax.adamw(learning_rate=0.01),
    ),
    root_dir=pathlib.Path(
        'root',
        'last',
        'append',
    ),
)

Caveat:

* Json do no preserve tuple/list information. `tuple` are restored as `list`